# Train Adapt-LLM on few examples

In [1]:
#pip install peft

In [2]:
#%pip uninstall peft -y
#%pip install git+https://github.com/huggingface/peft
#%pip install git+https://github.com/huggingface/peft.git@e536616888d51b453ed354a6f1e243fecb02ea08

In [3]:
 !rm -rf /kaggle/working/*

In [4]:
#%pip install git+https://github.com/huggingface/peft
%pip install -U bitsandbytes
%pip install -U transformers
%pip install -U peft
%pip install -U accelerate
%pip install -U trl 
#%pip install https://pypi.org/simple/ bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 12.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 637.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.7/401.7 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 81.1 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.22.2
    Uninstalling huggingface-hub-0.22.2:
      Successfully uninstalled huggingface-hub-0.22.2
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.39.3
    Uninstalling transformers-4.39.3:
      Successfully uninstalled transformers-4.3

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments, BitsAndBytesConfig,HfArgumentParser,pipeline, logging
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import os,torch, wandb
from datasets import DatasetDict
from trl import SFTTrainer, SFTConfig
import accelerate
import pandas as pd

2024-06-06 08:32:18.397084: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-06 08:32:18.397185: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-06 08:32:18.567023: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [6]:
model = "AdaptLLM/finance-chat"

# Load base model
bnb_config = BitsAndBytesConfig(  
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= False,
)
model_upd = AutoModelForCausalLM.from_pretrained(
        model,
        quantization_config=bnb_config,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        trust_remote_code=True,
)
model_upd.config.use_cache = False # silence the warnings. Please re-enable for inference!
model_upd.config.pretraining_tp = 1
model_upd.gradient_checkpointing_enable()

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model, trust_remote_code=True)
tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True
tokenizer.add_bos_token, tokenizer.add_eos_token

config.json:   0%|          | 0.00/635 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/28.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/9.88G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/9.89G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/7.18G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/849 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

(True, True)

In [7]:
### Adding the adapters in the layers
model_upd = prepare_model_for_kbit_training(model_upd)
peft_config = LoraConfig(
    lora_alpha=512,
    lora_dropout=0.1,
    r=512,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj","gate_proj"]
)
model_upd = get_peft_model(model_upd, peft_config)

In [8]:
#Hyperparamter
training_arguments = SFTConfig(
    output_dir="./results",
    num_train_epochs=70, #100 #40
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=425 ,#840,#600 #240,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant"
)

In [9]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(model_upd))

trainable model parameters: 784334848
all model parameters: 4284755968
percentage of trainable model parameters: 18.31%


## Tokenization

In [10]:
## GHOST
'''
data=pd.read_csv("/kaggle/input/update-gh/Fraud_Ghost_end.csv")
test_data=pd.read_csv("/kaggle/input/ghossttt/Test_Fraud_Ghost_end.csv")

#data = data.drop(data.columns[0], axis=1)
#test_data = test_data.drop(test_data.columns[0], axis=1)

json_data = data.to_json(orient='records')
json_test_data = test_data.to_json(orient='records')
'''

'\ndata=pd.read_csv("/kaggle/input/update-gh/Fraud_Ghost_end.csv")\ntest_data=pd.read_csv("/kaggle/input/ghossttt/Test_Fraud_Ghost_end.csv")\n\n#data = data.drop(data.columns[0], axis=1)\n#test_data = test_data.drop(test_data.columns[0], axis=1)\n\njson_data = data.to_json(orient=\'records\')\njson_test_data = test_data.to_json(orient=\'records\')\n'

In [11]:
## Misclassification
'''
data=pd.read_csv("/kaggle/input/misclasss/New_train_M.csv")
test_data=pd.read_csv("/kaggle/input/misclasss/New_test_M.csv.csv")

#data = data.drop(data.columns[0], axis=1)
#test_data = test_data.drop(test_data.columns[0], axis=1)

json_data = data.to_json(orient='records')
json_test_data = test_data.to_json(orient='records')
'''

'\ndata=pd.read_csv("/kaggle/input/misclasss/New_train_M.csv")\ntest_data=pd.read_csv("/kaggle/input/misclasss/New_test_M.csv.csv")\n\n#data = data.drop(data.columns[0], axis=1)\n#test_data = test_data.drop(test_data.columns[0], axis=1)\n\njson_data = data.to_json(orient=\'records\')\njson_test_data = test_data.to_json(orient=\'records\')\n'

In [12]:
## Timesheet

data=pd.read_csv("/kaggle/input/timesheet-upda/Timesheet_100_input_anonym.csv")
test_data=pd.read_csv("/kaggle/input/timesheet-upda/Timesheet_test_30_anonym.csv")

#data = data.drop(data.columns[0], axis=1)
#test_data = test_data.drop(test_data.columns[0], axis=1)

json_data = data.to_json(orient='records')
json_test_data = test_data.to_json(orient='records')


In [13]:
L=json_data.split('},{')
L_test = json_test_data.split('},{')

L[0]=L[0][2:]
L[-1]=L[-1][:-2]

L_test[0]=L_test[0][2:]
L_test[-1]=L_test[-1][:-2]

len(L)
#len(L_test)

94

In [14]:
def formatList(L):
    L_res=[]
    i=0

    while i < len(L) : 
        strr="{ " + L[i] + " },"
        j=i+1
        while (j< len(L)) and (L[i].split(",")[0]== L[j].split(",")[0]): 
            strr+= "{ " + L[j].split('"Explanation":')[0] + " },"
            j+=1
           
        L_res.append(strr + ' "Explanation":' + L[j-1].split('"Explanation":')[1])
        print(len(L_res))
        i=j
        
    return L_res

In [ ]:
L_format = formatList(L)
L_test_format = formatList(L_test)

In [16]:
for i in range(len(L_format)) : 
    Scenario = L_format[i].split('"Explanation":"')
    L_format[i]= Scenario[0] + '\n "Explanation":"' + Scenario[1]

for i in range(len(L_test_format)) : 
    Scenario = L_test_format[i].split('"Explanation":"')
    L_test_format[i]= Scenario[0] + '\n "Explanation":"' + Scenario[1]
    

In [18]:
# Timesheet

for i in range(len(L_format)) : 
    L_format[i]="Generate a Timesheet Fraud Scenario : { " + L_format[i] + " },"

for i in range(len(L_test_format)) : 
    L_test_format[i]="Generate a Timesheet Fraud Scenario : { " + L_test_format[i] + " },"
    
L_test_format[-1]
L_format[-1]

'Generate a Timesheet Fraud Scenario : { { "PersonID":864341,"SOURCE":"R","Payroll_Period":"12\\/2023","Absence_Reason":"Sick Leave","Absence_CODE":"MAL","Absence_Start_Date":"01\\/12\\/2023","Absence_End_Date":"01\\/12\\/2023","Absence_Duration":4,"Acquired_Leave_days":"16.5","Taken_Leave_days":"4","Remaining_Leave_days":"12","Following_year_reported_Leave":0.0,"Explanation":null },{ "PersonID":864341,"SOURCE":"R","Payroll_Period":"12\\/2023","Absence_Reason":"Sick Leave","Absence_CODE":"MAL","Absence_Start_Date":"04\\/12\\/2023","Absence_End_Date":"08\\/12\\/2023","Absence_Duration":5,"Acquired_Leave_days":"12","Taken_Leave_days":"5","Remaining_Leave_days":"7","Following_year_reported_Leave":0.0, },{ "PersonID":864341,"SOURCE":"R","Payroll_Period":"12\\/2023","Absence_Reason":"Sick Leave","Absence_CODE":"MAL","Absence_Start_Date":"11\\/12\\/2023","Absence_End_Date":"13\\/12\\/2023","Absence_Duration":3,"Acquired_Leave_days":"10.42","Taken_Leave_days":"3","Remaining_Leave_days":"7","F

In [19]:
# GHOST

'''
for i in range(len(L_format)) : 
    L_format[i]="Generate a Ghost Fraud Scenario : { " + L_format[i] + " },"

for i in range(len(L_test_format)) : 
    L_test_format[i]="Generate a Ghost Fraud Scenario : { " + L_test_format[i] + " },"
    
L_test_format[2]
'''

'\nfor i in range(len(L_format)) : \n    L_format[i]="Generate a Ghost Fraud Scenario : { " + L_format[i] + " },"\n\nfor i in range(len(L_test_format)) : \n    L_test_format[i]="Generate a Ghost Fraud Scenario : { " + L_test_format[i] + " },"\n    \nL_test_format[2]\n'

In [20]:
# Misclassification

'''
for i in range(len(L_format)) : 
    L_format[i]="Generate a Misclassification Fraud Scenario : { " + L_format[i] + " },"

for i in range(len(L_test_format)) : 
    L_test_format[i]="Generate a Misclassification Fraud Scenario : { " + L_test_format[i] + " },"
    
L_test_format[2]
'''

'\nfor i in range(len(L_format)) : \n    L_format[i]="Generate a Misclassification Fraud Scenario : { " + L_format[i] + " },"\n\nfor i in range(len(L_test_format)) : \n    L_test_format[i]="Generate a Misclassification Fraud Scenario : { " + L_test_format[i] + " },"\n    \nL_test_format[2]\n'

### Prepare data format

In [21]:
import pyarrow as pa
import pyarrow.dataset as ds
import pandas as pd
from datasets import Dataset

#dataset = ds.dataset(pa.Table.from_pandas(data).to_batches())

### convert to Huggingface dataset
#hg_dataset = Dataset(pa.Table.from_pandas(data))

In [22]:
d = pd.DataFrame(L_format)
dataset = ds.dataset(pa.Table.from_pandas(d).to_batches())
### convert to Huggingface dataset
hg_dataset = Dataset(pa.Table.from_pandas(d))

test_d = pd.DataFrame(L_test_format)
test_dataset = ds.dataset(pa.Table.from_pandas(test_d).to_batches())
### convert to Huggingface dataset
test_hg_dataset = Dataset(pa.Table.from_pandas(test_d))

In [23]:
#test_hg_dataset

In [24]:
#hg_dataset

## Train model

In [25]:
'''
from transformers import EarlyStoppingCallback

# Define early stopping parameters
early_stopping_callback = EarlyStoppingCallback(
    early_stopping_patience=3,  # Number of evaluations with no improvement before stopping
    load_best_model_at_end = True
)

training_arguments.evaluation_strategy = "steps"
training_arguments.eval_steps = 25  
'''

'\nfrom transformers import EarlyStoppingCallback\n\n# Define early stopping parameters\nearly_stopping_callback = EarlyStoppingCallback(\n    early_stopping_patience=3,  # Number of evaluations with no improvement before stopping\n    load_best_model_at_end = True\n)\n\ntraining_arguments.evaluation_strategy = "steps"\ntraining_arguments.eval_steps = 25  \n'

In [26]:
# Setting sft parameters
trainer = SFTTrainer(
    model=model_upd,
    train_dataset=hg_dataset,
    eval_dataset=test_hg_dataset,
    peft_config=peft_config,
    max_seq_length= None,
    dataset_text_field="0",
    tokenizer=tokenizer,
    args=training_arguments,
    packing= False
)

/opt/conda/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:276: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:305: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/34 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/15 [00:00<?, ? examples/s]

In [27]:
wandb.login(key='')

# Initialize a new wandb run
wandb.init(project="adapt-timesheet-1", name="run-7")

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ouesleti-farah (farah_o). Use `wandb login --relogin` to force relogin
wandb: wandb version 0.17.0 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.16.6
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240606_083705-fgbvm3hu
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run run-7
wandb: ⭐️ View project at https://wandb.ai/farah_o/adapt-timesheet-1
wandb: 🚀 View run at https://wandb.ai/farah_o/adapt-timesheet-1/runs/fgbvm3hu


In [28]:
#import torch
#torch.cuda.empty_cache()

In [29]:
training_output = trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
25,0.288900
50,0.065300
75,0.041100
100,0.032200
125,0.041000
150,0.024600
175,0.031300
200,0.030200
225,0.030500
250,0.023600


/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


## Pushing model to huggingface

In [30]:
#access token
hg_token= ""
import huggingface_hub
huggingface_hub.login(hg_token)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [31]:
trainer.save_model('adapt-llm-Timesheet-Fr-70xr512')

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [32]:
#model_upd.push_to_hub("adapt-llm-Misc-Fr")

In [33]:
#from huggingface_hub import login
#login()

In [34]:
from huggingface_hub import HfApi, create_repo
api = HfApi()

api.create_repo(repo_id="adapt-llm-Timesheet-Fr-70xr512")

RepoUrl('https://huggingface.co/FO-UA/adapt-llm-Timesheet-Fr-70xr512', endpoint='https://huggingface.co', repo_type='model', repo_id='FO-UA/adapt-llm-Timesheet-Fr-70xr512')

In [35]:
api.upload_folder(    
    folder_path="./adapt-llm-Timesheet-Fr-70xr512",
    repo_id="FO-UA/adapt-llm-Timesheet-Fr-70xr512",
    repo_type="model"
)

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.37k [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/3.14G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/FO-UA/adapt-llm-Timesheet-Fr-70xr512/commit/36e6507fb7ac4737b28ad77b018b605d42ac8f70', commit_message='Upload folder using huggingface_hub', commit_description='', oid='36e6507fb7ac4737b28ad77b018b605d42ac8f70', pr_url=None, pr_revision=None, pr_num=None)